In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# Set GPU configuration to use NVIDIA CUDA devices
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
import tensorflow as tf

# List all physical devices
physical_devices = tf.config.list_physical_devices()
print("All Physical Devices:", physical_devices)

# List only GPU devices
gpus = tf.config.list_physical_devices('GPU')
print("GPUs Available:", gpus)


All Physical Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:


# Load or distribute your data here
home = r'mnt/e/BigRun'
#resolution = home + 'TFDL_BigRunWS_V5_TS_500'
train_file = r"/mnt/e/BigRun/train/BigRunWS_V5_T_500_train_part_1.csv"
test_file = r"/mnt/e/BigRun/test/BigRunWS_V5_T_500_test_part_1.csv"

train_data_full = pd.read_csv(train_file, usecols=['PageName', 'Clay', 'Sand', 'Silt', 'Elevation', 'Slope', 'Aspect', 'MODIS', 'Smerge', 'Date', 'LAI', 'ALB', 'Temp'], engine='pyarrow')
test_data_full = pd.read_csv(test_file, usecols=['PageName', 'Clay', 'Sand', 'Silt', 'Elevation', 'Slope', 'Aspect', 'MODIS', 'Smerge', 'Date', 'LAI', 'ALB', 'Temp', 'AHRR'], engine='pyarrow')

train_data = train_data_full[['Clay', 'Sand', 'Silt', 'Elevation', 'Aspect', 'Slope', 'MODIS', 'Smerge', 'Date', 'LAI', 'ALB', 'Temp']].dropna()
test_data = test_data_full[['Clay', 'Sand', 'Silt', 'Elevation', 'Aspect', 'Slope', 'MODIS', 'Smerge', 'Date', 'LAI', 'ALB', 'Temp']].dropna()

modis = test_data_full['MODIS']
dates = test_data_full['Date']
ahrr = test_data_full['AHRR']
test_page = test_data_full[['PageName']]

# Convert Date to a numerical value
test_data['Date'] = pd.to_datetime(test_data['Date'], format="%Y-%m-%d").astype(int)
train_data['Date'] = pd.to_datetime(train_data['Date'], format="%Y-%m-%d").astype(int)

# Split features and labels
X_train = train_data[['Clay', 'Sand', 'Silt', 'Elevation', 'Aspect', 'Slope', 'MODIS', 'LAI', 'ALB', 'Temp', "Date"]]
y_train = train_data['Smerge']
X_test = test_data[['Clay', 'Sand', 'Silt', 'Elevation', 'Aspect', 'Slope', 'MODIS', 'LAI', 'ALB', 'Temp', "Date"]]
y_test = test_data['Smerge']

# Standardize features



In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:

# Define the model
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Build and train the model
model = build_model()
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Predict on the test dataset
pred = model.predict(X_test)

# Save predictions
test_data['ML_'] = pred
test_data['MODIS'] = modis
test_data['Date'] = dates
test_data['PageName'] = test_page
test_data['AHRR'] = ahrr
test_data.to_csv("mnt/e/BigRun/TFDL_BigRunWS_V5_TS_500.csv", index=False)

/home/asancchez/.local/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1733624516.048856     454 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9536 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/50


I0000 00:00:1733624518.064782     856 service.cc:148] XLA service 0x7fa35000cb90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733624518.065213     856 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-12-07 20:21:58.087176: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733624518.164328     856 cuda_dnn.cc:529] Loaded cuDNN version 90600


   101/372339 ━━━━━━━━━━━━━━━━━━━━ 9:28 2ms/step - loss: 0.0084 - mae: 0.0699 

I0000 00:00:1733624519.154385     856 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


372339/372339 ━━━━━━━━━━━━━━━━━━━━ 596s 2ms/step - loss: 0.0013 - mae: 0.0272
Epoch 2/50
372339/372339 ━━━━━━━━━━━━━━━━━━━━ 603s 2ms/step - loss: 0.0011 - mae: 0.0253
Epoch 3/50
372339/372339 ━━━━━━━━━━━━━━━━━━━━ 601s 2ms/step - loss: 0.0011 - mae: 0.0249
Epoch 4/50
372339/372339 ━━━━━━━━━━━━━━━━━━━━ 576s 2ms/step - loss: 0.0011 - mae: 0.0246
Epoch 5/50
359636/372339 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 0.0011 - mae: 0.0243

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is available: ", gpus)
else:
    print("No GPU detected")


GPU is available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
test_data_full_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TFDL_BigRunWS_V5_TS_500.csv", engine='pyarrow')
test_file = '/content/drive/MyDrive/Colab Notebooks/BigRunWS_V5_L_500_test_part_10.csv'
test_data_full = pd.read_csv(test_file, engine='pyarrow')

In [ ]:
test_data_full['ML_'] = test_data_full_1['ML_']
print(test_data_full)

        PageName        Date      Aspect       Clay   Elevation       Sand  \
0          EL203  2009-05-07  181.281639   5.761240  535.269261  83.761319   
1          ES456  2009-06-20  222.426775  40.332804  418.311052   4.703199   
2          HZ272  2016-09-08  144.423113  12.902163  464.717362  68.472239   
3          DD319  2012-06-01  215.834175   4.937830  481.469028  87.231289   
4          RY566  2009-08-25  135.303861  16.521910  274.776140  52.549591   
...          ...         ...         ...        ...         ...        ...   
2172342    HT281  2015-06-03  162.454179  17.897157  481.203375  52.336996   
2172343    SD641  2009-06-02  160.167098  32.830077  196.057226  14.691847   
2172344    IU307  2015-04-25  163.371199  22.945723  424.049061  46.298830   
2172345    EZ152  2009-05-25  157.423177  14.929941  587.792622  64.321923   
2172346    GT482  2012-05-03  195.322385  29.674426  399.328221  13.258646   

              Silt     Slope    Smerge      ALB    LAI       Te

In [ ]:
test_data_full.to_csv("/content/drive/MyDrive/Colab Notebooks/TFDL_BigRunWS_V5_TS_500_2.csv", index=False)

In [ ]:
!nvidia-smi

Tue Nov 26 19:43:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   43C    P8              16W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import tensorflow as tf

# Create a simple model for testing
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generate dummy data to use with the model
import numpy as np

x_train = np.random.random((10000, 784))
y_train = np.random.randint(10, size=(10000,))

# Train the model (this will utilize the GPU if available)
model.fit(x_train, y_train, epochs=5, batch_size=64)
